In [2]:
!cat local-volumes.yaml

apiVersion: v1
kind: PersistentVolume
metadata:
  name: local-pv-1
  labels:
    type: local
spec:
  capacity:
    storage: 20Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: /tmp/data/pv-1
---
apiVersion: v1
kind: PersistentVolume
metadata:
  name: local-pv-2
  labels:
    type: local
spec:
  capacity:
    storage: 20Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: /tmp/data/pv-2


In [3]:
!kubectl get pods

No resources found.


In [6]:
!kubectl get deployments

No resources found.


In [4]:
!minikube status

minikube: Running
cluster: Running
kubectl: Correctly Configured: pointing to minikube-vm at 192.168.99.101


In [7]:
!kubectl create -f local-volumes.yaml

persistentvolume/local-pv-1 created
persistentvolume/local-pv-2 created


In [8]:
!kubectl get persistentVolumes

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM                    STORAGECLASS   REASON    AGE
local-pv-1                                 20Gi       RWO            Retain           Available                                                     1m
local-pv-2                                 20Gi       RWO            Retain           Available                                                     1m
pvc-640cb47f-b5d7-11e8-8b55-080027c0edcf   20Gi       RWO            Delete           Released    default/wp-pv-claim      standard                 21h
pvc-8d320f09-b5d7-11e8-8b55-080027c0edcf   20Gi       RWO            Delete           Released    default/mysql-pv-claim   standard                 21h


In [9]:
!kubectl apply -f mysql-deployment.yaml

service/wordpress-mysql created
persistentvolumeclaim/mysql-pv-claim created
deployment.apps/wordpress-mysql created


In [11]:
!kubectl apply -f wordpress-deployment.yaml

service/wordpress created
persistentvolumeclaim/wp-pv-claim created
deployment.apps/wordpress created


In [12]:
!kubectl get pods

NAME                               READY     STATUS    RESTARTS   AGE
wordpress-6599c4d6bc-sxgjf         1/1       Running   0          10s
wordpress-mysql-764f7c5f65-v4xnh   1/1       Running   0          52s


In [13]:
!cat wordpress-deployment.yaml

apiVersion: v1
kind: Service
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  ports:
    - port: 80
  selector:
    app: wordpress
    tier: frontend
  type: LoadBalancer
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: wp-pv-claim
  labels:
    app: wordpress
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
---
apiVersion: apps/v1beta2 # for versions before 1.8.0 use apps/v1beta1
kind: Deployment
metadata:
  name: wordpress
  labels:
    app: wordpress
spec:
  selector:
    matchLabels:
      app: wordpress
      tier: frontend
  strategy:
    type: Recreate
  template:
    metadata:
      labels:
        app: wordpress
        tier: frontend
    spec:
      containers:
      - image: wordpress:4.8-apache
        name: wordpress
        env:
        - name: WORDPRESS_DB_HOST
          value: wordpress-mysql
        - name: WORDPRESS_DB_PASSWORD
          value: PASSWORDS_IN_PLAIN_TEXT_ARE_BAD_WE_WILL_SHOW_SOMETHING

In [14]:
!cat mysql-deployment.yaml

apiVersion: v1
kind: Service
metadata:
  name: wordpress-mysql
  labels:
    app: wordpress
spec:
  ports:
    - port: 3306
  selector:
    app: wordpress
    tier: mysql
  clusterIP: None
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mysql-pv-claim
  labels:
    app: wordpress
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi
---
apiVersion: apps/v1beta2 # for versions before 1.8.0 use apps/v1beta1
kind: Deployment
metadata:
  name: wordpress-mysql
  labels:
    app: wordpress
spec:
  selector:
    matchLabels:
      app: wordpress
      tier: mysql
  strategy:
    type: Recreate
  template:
    metadata:
      labels:
        app: wordpress
        tier: mysql
    spec:
      containers:
      - image: mysql:5.6
        name: mysql
        env:
        - name: MYSQL_ROOT_PASSWORD
          value: PASSWORDS_IN_PLAIN_TEXT_ARE_BAD_WE_WILL_SHOW_SOMETHING_MORE_SECURE_LATER
        ports:
        - containerPort: 3306
          name:

In [15]:
!minikube stop

Stopping local Kubernetes cluster...
Machine stopped.


In [16]:
!minikube start

Starting local Kubernetes v1.10.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [17]:
!minikube service wordpress --url

http://192.168.99.101:31138


if we insert the following address in browser we can install wordpress

<font color=red>Now our wordpress instance is up and running. We can delete the deployment after adding some content to it. If the create the deployment again, we dont loose the content from the last deployment, as we have used persistent volumes for both myaql and wordpress</font>

#### Some of the commands learned in this notebook

    kubectl create -f local-volumes.yaml
    kubectl apply -f mysql-deployment.yaml
    kubectl apply -f wordpress-deployment.yaml
    minikube service wordpress --url

#### Cleanup

In [1]:
%%bash
kubectl delete -f ./mysql-deployment.yaml
kubectl delete -f ./wordpress-deployment.yaml
minikube stop
minikube start

service "wordpress-mysql" deleted
persistentvolumeclaim "mysql-pv-claim" deleted
deployment.apps "wordpress-mysql" deleted
service "wordpress" deleted
persistentvolumeclaim "wp-pv-claim" deleted
deployment.apps "wordpress" deleted
Stopping local Kubernetes cluster...
Machine stopped.
Starting local Kubernetes v1.10.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.
